In [1]:
import os
from os.path import isfile
import pandas as pd
import pickle
from main_effect import main_effect
from compile_studies import compile_studies
from contribution import contribution
from compute_contrast import compute_contrast
%matplotlib inline

cwd = os.getcwd()
raw_folder = f'{cwd}/DataRaw/'
pickle_folder = f'{cwd}/DataPickle/'

'''filename = askopenfilename()
df = pd.read_excel(filename, engine='openpyxl', header=None)'''

meta_df = pd.read_excel(raw_folder + 'Metastocalculate.xlsx', engine='openpyxl', header=None)
with open(pickle_folder + 'experiments.pickle', 'rb') as f:
    exp_all, tasks = pickle.load(f)

In [2]:
for row_idx in range(meta_df.shape[0]):
    if meta_df.iloc[row_idx, 0] == 'M': #Main Effect Analysis
        exp_name = meta_df.iloc[row_idx, 1]
        exp_idxs = compile_studies(meta_df, row_idx, tasks)
        exp_df = exp_all.loc[exp_idxs].reset_index(drop=True)
        if len(exp_idxs) >= 12: 
            print(f'{exp_name} : {len(exp_idxs)} experiments; average of {exp_df.Subjects.mean():.2f} subjects per experiment')
            main_effect(exp_df, exp_name)
            contribution(exp_df, exp_name, exp_idxs, tasks)
        else:
            print(f"{exp_name} : only {len(exp_idxs)} experiments - not analyzed!")
            
    if meta_df.iloc[row_idx, 0] == 'C': # Contrast Analysis
        exp_name1 = meta_df.iloc[row_idx, 1]
        exp_idxs1 = compile_studies(meta_df, row_idx, tasks)
        exp_df1 = exp_all.loc[exp_idxs1].reset_index(drop=True)

        exp_name2 = meta_df.iloc[row_idx + 1, 1]
        exp_idxs2 = compile_studies(meta_df, row_idx+1, tasks)
        exp_df2 = exp_all.loc[exp_idxs2].reset_index(drop=True)
        
        if len(exp_idxs1) >= 12 and len(exp_idxs) >= 12:
            if not isfile(f"{cwd}/ALE/Results/{exp_name1}_cFWE05.nii"):
                main_effect(exp_df1, exp_name1)
                contribution(exp_df1, exp_name1, exp_idxs1, tasks)
            if not isfile(f"{cwd}/ALE/Results/{exp_name2}_cFWE05.nii"):
                main_effect(exp_df2, exp_name2)
                contribution(exp_df2, exp_name2, exp_idxs2, tasks)
                
            for i in reversed(exp_idxs1):
                if i in exp_idxs2:
                    exp_idxs1.remove(i)
                    exp_idxs2.remove(i)
                    
            exp_df1 = exp_all.loc[exp_idxs1].reset_index(drop=True)
            exp_df2 = exp_all.loc[exp_idxs2].reset_index(drop=True)
            compute_contrast(exp_df1, exp_name1, exp_df2, exp_name2)
                
            

Other : 19 experiments; average of 22.26 subjects per experiment
Other - loading Foci
Other - loading ALE
Other - loading null PDF
Other - loading p-values & TFCE
Other - loading noise
Other - done!
Cwclassic : 37 experiments; average of 25.76 subjects per experiment
Cwclassic - loading Foci
Cwclassic - loading ALE
Cwclassic - loading null PDF
Cwclassic - computing p-values & TFCE
Cwclassic - simulating noise


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.


KeyboardInterrupt: 